In [ ]:
import pandas as pd
import os
from tabulate import tabulate

# Configuração do caminho do arquivo
caminho = r'C:\Users\Caio\Documents\Programas\data.csv'

if os.path.exists(caminho):
    try:
        # Tentar ler com diferentes codificações
        codificacoes = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
        df = None
        
        for cod in codificacoes:
            try:
                # Tentar ler o arquivo CSV com diferentes separadores
                try:
                    df = pd.read_csv(caminho, encoding=cod)
                except pd.errors.ParserError:
                    df = pd.read_csv(caminho, sep=';', encoding=cod)
                break
            except UnicodeDecodeError:
                continue
        
        if df is None:
            print("\nERRO: Não foi possível ler o arquivo com as codificações testadas")
            print("Tente abrir o arquivo em um editor de texto para verificar a codificação correta\n")
        else:
            # Pré-processamento dos dados
            df.fillna('N/A', inplace=True)
            
            # Verificar colunas necessárias
            required_columns = ['Description', 'Country', 'Quantity']
            missing_cols = [col for col in required_columns if col not in df.columns]
            
            if missing_cols:
                print("\nERRO: Colunas necessárias não encontradas no arquivo")
                print(f"Colunas encontradas: {list(df.columns)}")
                print(f"Colunas faltantes: {missing_cols}\n")
            else:
                # Filtrar produtos válidos (excluindo 'N/A')
                df_filtered = df[df['Description'] != 'N/A'].copy()
                
                # Garantir que Quantity seja numérico
                df_filtered['Quantity'] = pd.to_numeric(df_filtered['Quantity'], errors='coerce')
                df_filtered = df_filtered.dropna(subset=['Quantity'])
                
                # 1. Top 10 produtos mais vendidos (por quantidade total)
                top_products = (df_filtered.groupby('Description')['Quantity']
                              .sum()
                              .sort_values(ascending=False)
                              .head(10))
                
                # 2. País que mais comprou cada produto (por volume total)
                def get_top_country(group):
                    return (group.groupby('Country')['Quantity']
                            .sum()
                            .idxmax())
                
                country_by_product = (df_filtered[df_filtered['Description']
                                    .isin(top_products.index)]
                                    .groupby('Description')
                                    .apply(get_top_country))
                
                # 3. Quantidade média por produto (inteiro)
                avg_quantity = (df_filtered.groupby('Description')['Quantity']
                               .mean()
                               .astype(int))
                
                # 4. Número de pedidos por produto
                order_count = df_filtered['Description'].value_counts()
                
                # Criar tabela de resultados completa
                results = pd.DataFrame({
                    'Produto': top_products.index,
                    'Total Vendido (Unidades)': top_products.values,
                    'País Principal': [country_by_product[prod] for prod in top_products.index],
                    'Quantidade Média/Pedido': [avg_quantity[prod] for prod in top_products.index],
                    'Número de Pedidos': [order_count[prod] for prod in top_products.index]
                }).reset_index(drop=True)
                
                # Formatar e exibir resultados
                print("\n" + "="*100)
                print("RELATÓRIO: TOP 10 PRODUTOS MAIS VENDIDOS POR VOLUME".center(100))
                print("="*100 + "\n")
                print(tabulate(results, 
                             headers='keys', 
                             tablefmt='pretty', 
                             showindex=False,
                             numalign="right",
                             stralign="left"))
                print("\n" + "="*100)
                print(f"Total de registros válidos analisados: {len(df_filtered)}".center(100))
                print("="*100)
                
                # Salvar resultados em CSV e Excel
                results.to_csv('top10_produtos_mais_vendidos.csv', index=False, encoding='utf-8-sig')
                results.to_excel('top10_produtos_mais_vendidos.xlsx', index=False)
                print("\nResultados salvos em:")
                print("- top10_produtos_mais_vendidos.csv")
                print("- top10_produtos_mais_vendidos.xlsx")
                
    except Exception as e:
        print(f"\nERRO: {str(e)}\n")
else:
    print(f"\nERRO: Arquivo não encontrado em: {caminho}\n")